0. 自增序列实现

In [ ]:
# def addChunkData(self,documents):
#     rt_ids=[]
#     with dbmongo.Mongo("vector") as vdb:
#         db = vdb.db
#         chunkset = db['chunk'] #集合
#         counters = db['counters'] #序列集合
#         for document in documents:
#             result = counters.find_one_and_update(
#                 {"chunk_id": "chunk"},# chunk 是集合 chunk_id 序列名字
#                 {"$inc": {"sequence_value": 1}},
#                 upsert=True,
#                 return_document=True
#             )
#             sequence_value = result["sequence_value"] #自增序列
#             document['chunk_id'] = sequence_value
#             # rtd = chunkset.insert_one(document)
#             # print("rt_ids:",rtd.inserted_id)
#             rt_ids.append(sequence_value)
#             print("sequence_value:",sequence_value)
#         rtd = chunkset.insert_many(documents)
#         if len(rtd.inserted_ids)>0:
#             return rt_ids
#         # db[collection_name].insert_one(document)
#     return rt_ids


1. 检查textdb内document.chunk_ids和chunk.chunk_id是否对应

In [2]:
import pymongo
from collections import defaultdict
client = pymongo.MongoClient('mongodb://localhost:27017/')
mydb = client['textdb']
document = list(mydb['document'].find({}, {"_id":0, "document_str_id":1, "chunk_ids":1}))
print(len(document))
document_dict = {each["document_str_id"]:each["chunk_ids"] for each in document}
chunk = list(mydb['chunk'].find({}, {"_id":0, "chunk_id":1, "document_str_id":1}))
print(len(chunk))
chunk_dict = defaultdict(list)
for each in chunk:
    chunk_dict[each["document_str_id"]].append(each["chunk_id"])

8016
107779


In [16]:
for key in document_dict.keys():
    if set(document_dict[key]) != set(chunk_dict[key]):
        print(key)

for key in chunk_dict.keys():
    try:
        if set(document_dict[key]) != set(chunk_dict[key]):
            print(key)
    except Exception as e:
        print("Exception! ", e)
        print(key)

Exception!  'sA_3978866'
sA_3978866


以下是关于'sA_3978866'

In [12]:
list(mydb['company'].find({"company_id":2}))

[{'_id': ObjectId('6568ad60093d9d71557a0da5'),
  'windcode': 'MSFT.O',
  'company_name_en': 'microsoft',
  'company_name_cn': '微软',
  'company_id': 2}]

In [7]:
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017/')
mydb = client['textdb']
chunk_ids = list(mydb['document'].find({}, {"chunk_ids":1, "document_str_id":1, "company_id":1}))
for each in chunk_ids:
    for i in range(18666, 18672):
        if i in each['chunk_ids']:
            print(each['document_str_id'], each['company_id'])

for i in range(18668, 18671):
    print(1)

cnt = 0
for each in chunk_ids:
    cnt += len(each['chunk_ids'])
print(cnt)

chunk_id = list(mydb['chunk'].find({}, {"chunk_id":1}))
print(len(chunk_id))

sA_3978529 2
cicc_325515_1151069 2
cicc_325515_1151069 2
1
1
1
107776
107779


In [11]:
for i in range(18668, 18671):
    ch = list(mydb['chunk'].find({"chunk_id":i}, {}))
    print(ch[0]['chunk_text'][:20])

In the stock market,
Pete Callahan, said,
a capital intensive 


In [4]:
list(mydb['document'].find({"document_str_id":"sA_3978866"}))

[]

In [5]:
ch = list(mydb['chunk'].find({"document_str_id":"sA_3978866"}))
for i in ch:
    print(i)

{'_id': ObjectId('6569576c093d9d71557c628b'), 'document_str_id': 'sA_3978866', 'chunk_text': 'In the stock market, there is one new game in town and then there is everything else, according to the head of hedge fund coverage for Goldman Sachs. Thanks in large part to an AI boost the biggest stocks have almost single-handedly dragged the S&P 500 (SP500) (SPY) (VOO) (IVV) into a technical bull market. "AI has taken off like a rocket and drawn all kinds of investors in - this is a simple theme that retail traders can easily attach to," Tony Pasquariello wrote in a note. The Goldman Sachs AI Basket is up more than 50% year to date. "In that context, recall the wisdom of a great tech investor: \'positive earnings surprises occur when revenue and earnings growth are accelerating, when average selling prices are rising, and when gross margin and operating margin are rising,\'" Pasquariello said. Perception "may be reality for many companies at this stage of the A.I.," Goldman tech specialist 

In [3]:
from tqdm import tqdm
chunk_ids_alone = list(range(18668, 18671))
doc_str_id_and_chunk_ids = list(mydb['document'].find({}, {"chunk_ids":1, "document_str_id":1}))
flag = 0
not_in_doc_chunk_ids = []
for chunk_id in tqdm(chunk_ids_alone):
    for str_id_and_chunk_id in doc_str_id_and_chunk_ids:
        if chunk_id > str_id_and_chunk_id["chunk_ids"][-1]:
            continue
        if chunk_id < str_id_and_chunk_id["chunk_ids"][0]:
            continue
        if chunk_id in str_id_and_chunk_id["chunk_ids"]:
            flag = 1
    if not flag:
        not_in_doc_chunk_ids.append(chunk_id)
    flag = 0
print(not_in_doc_chunk_ids)

100%|██████████| 3/3 [00:00<00:00, 640.61it/s]

[18668, 18669, 18670]


2. 启动textdb，在test当中进行测试

In [1]:
from textdb import *
import json
with open("/root/mwy/Simple-RAG-LLM/product/config.json", "r") as f:
    config = json.load(f)
db = TextDB(config)

Starting Text DB service ...
- Done


In [2]:
db._mydb.list_collection_names()

['document',
 'datatype',
 'copyright',
 't',
 'company',
 'chunk',
 'chat',
 'counters']

In [2]:
db.insert_copyright("jpmorgan")

2

In [4]:
db.get_copyright_ids(["jpmorgan", "bernstein"])

[2, 1]

In [6]:
db._copyright_name_to_id

{'bernstein': 1, 'jpmorgan': 2}

In [2]:
print(db.get_company_id_range())

(1, 1)


In [3]:
db.get_number_of_chunks()

3

2. 删

In [ ]:
db.remove_all_data()

3. 增

In [17]:
chunk_ids=[0, 1, 2] 
document_ids='test_id'
chunk_texts=["...", "...", "..."] 
embeddings=[[1, 2], [2, 3], [3, 2]]
db.insert_batch_chunks(document_ids,chunk_texts,embeddings)

[7, 8, 9]

In [4]:
db.insert_company("windco","testcompany", "测试公司")

1

In [3]:
db.insert_document( "test_id", "题目", 1, 1, 10, 202012, "测试的文章", [6, 6, 6])

'test_id'

4. 查

In [4]:
db.get_company_id_by_windcode("windco")

1

In [6]:
db.get_company_id_by_name('测试公司')

1

In [7]:
db.get_datatype_id_by_name("research")

Exception: ! Error: datatype 'research' not found!

In [17]:
db.get_datatype_id_range()

resp: {'errorCode': '0000000', 'errorFlag': True, 'errorMsg': '', 'data': [{'datatype_id': 1}, {'datatype_id': 2}, {'datatype_id': 3}, {'datatype_id': 4}, {'datatype_id': 5}, {'datatype_id': 6}]}


(1, 6)

In [18]:
db.get_grouped_texts([7,8,9])

[{'title': '题目', 'texts': '.........', 'date': 10}]

In [8]:
db.get_embedding_by_doc_str_id("test_id")

[]

In [15]:
db.is_in_doc_str_id_set("test_id")

True

In [16]:
db.get_doc_str_id_set()

{'test_id'}